## <font color=red>Литература</font>

1. Лекции «Алгоритмы: построение, анализ и реализация на языке программирования Си» - Ворожцов А.В., Винокуров Н.А.. Выложена в канале **#edu_materials**

# <font color=blue>Деревья. Продолжение 2</font>

# <font color=blue>Балансировка двоичного дерева</font>

Количество операций, необходимое для того, чтобы найти элемент в дереве, связано с высотой дерева $H$. Поиск элемента требует $O(H)$ операций. Если дерево случайное, то его высота может быть равна количеству узлов дерева $N$, а в лучшем случае высота двоичного дерева - $\left\lceil\log N\right\rceil$. Чтобы высота дерева в худшем случае была логарифмической, применяют **балансировку**.

**Методы балансировки деревьев поиска**  — это алгоритмы выполнения операций добавления и удаления записей (insert и del), которые гарантируют, что при любой последовательности выполнения запросов высота $H$ дерева поиска будет ограничена сверху линейной функцией от логарифма числа N хранимых записей:

$$H < A \cdot log_2 N + B$$

где $A$ и $B$ - фиксированные константы.

**Теорема**

Если в двоичном дереве c $N$ узлами выполнено хотя бы одно из следующих условий:

а) для любого узла число узлов в правом и левом поддереве $N_r$ , $N_l$ отличаются не более чем на $1$:
  $$N_r \le N_l + 1,\quad\quad N_l \le N_r +1$$
  
б) для любого узла число узлов в правом и левом поддереве $N_r$ , $N_l$ удовлетворяют условиям
  $$N_r \le 2 N_l + 1,\quad\quad N_l \le 2 N_r +1$$
  
а) для любого узла высота правого и левого поддеревьев $H_r$ , $H_l$ отличаются не более чем на $1$:
  $$H_r \le H_l + 1,\quad\quad H_l \le H_r +1$$
  
то высота дерева не превосходит $A \cdot log_2 N + B$, где $A$ и $B$ – некоторые положительные
константы, не зависящие от $N$.

**Доказательства** приведены в [1]

**Идально сбалансированное дерево** — это двоичное дерево поиска, для которого с помощью специальных алгоритмов поддерживается свойство (а).

**АВЛ-дерево** — это двоичное дерево поиска, для которого с помощью специальных алгоритмов поддерживается свойство (в).

Высота деревьев поиска, удовлетворяющих условиям (а), (б), (в) ограничена сверху логарифмической функцией от числа узлов дерева, а именно:

а) $H \le \log_2 N$

б) $H \le 1.70951 \log_2 N + 1$

в) $H \le 1.4404 \log_2 N + 1$

## <font color=green>АВЛ-дерево</font>

Названо так в честь математиков Адельского-Вельского и Ландиса.

Пусть высота правого поддерева равна $(k+2)$, а высота левого поддерева - $k$.

<img src="images/tree_rotations.png" alt="Drawing" style="width: 800px">

### Упражнение 1. Правое вращение

Добавьте в узлы ссылки на родителей. Реализуйте правое вращение.

In [61]:
class BSTNode:
    def __init__(self, key, value, left=None, right=None, parent=None):
        self.value = value
        self.key = key
        self.left = left
        self.right = right
        self.parent = parent
        
    def __str__(self):
        t = (
            repr(self.key),
            repr(self.value),
            'None' if self.left is None else repr(self.left.key),
            'None' if self.right is None else repr(self.right.key),
            'None' if self.parent is None else repr(self.parent.key)
        )
        fs = '(key={}, value={}, left_key={}, right_key={}, parent_key={})'
        return self.__class__.__name__ + fs.format(*t)
    
    def is_left_child(self):
        if self is self.parent.left:
            return True
        return False
    
    def is_right_child(self):
        if self is self.parent.right:
            return True
        return False
        

class BST:
    def __init__(self):
        self.root = None
        
    def _insert(self, key, value, root):
        if key == root.key:
            root.value = value
        elif key < root.key:
            if root.left is None:
                root.left = BSTNode(key, value, parent=root)
            else:
                self._insert(key, value, root.left)
        else:
            if root.right is None:
                root.right = BSTNode(key, value, parent=root)
            else:
                self._insert(key, value, root.right)
        
    def insert(self, key, value):
        if self.root is None:
            self.root = BSTNode(key, value)
        else:
            self._insert(key, value, self.root)
            
    def _find(self, key, root):
        if root is None:
            return None
        if key == root.key:
            return root.value
        elif key < root.key:
            return self._find(key, root.left)
        else:
            return self._find(key, root.right)
            
    def find(self, key):
        return self._find(key, self.root)
    
    def _traverse(self, root):
        if root is not None:
            self._traverse(root.left)
            print(root.key, root.value)
            self._traverse(root.right)
    
    def traverse(self):
        self._traverse(self.root)
        
    def _in_order_traverse(self, root, level):
        if root is not None:
            print(' '*4*level + '{}'.format(root.key))
            self._in_order_traverse(root.right, level+1)
            self._in_order_traverse(root.left, level+1)
        else:
            print(' '*4*level + '-')
        
    def in_order_traverse(self):
        self._in_order_traverse(self.root, 0)
    
    def _rotate_left(self, node):
        r = node.right
        if r is None:
            raise ValueError("no right subtree")
        rl = r.left
        if rl is None:
            raise ValueError("no right-left subtree")
        p = node.parent
        
        if p is None:
            self.root = r
        else:
            if node.is_left_child():
                p.left = r
            else:
                p.right = r
        
        r.parent = p
        r.left = node
        
        node.parent = r
        node.right = rl
        
        rl.parent = node

In [63]:
import random

bst = BST()
for _ in range(20):
    i = random.randint(0, 1000)
    bst.insert(i, str(i))
    
print(bst.root)
bst.in_order_traverse()

bst._rotate_left(bst.root.left)    

bst.in_order_traverse()

BSTNode(key=405, value='405', left_key=199, right_key=891, parent_key=None)
405
    891
        938
            -
            -
        703
            765
                790
                    -
                    -
                764
                    -
                    739
                        -
                        -
            564
                569
                    -
                    -
                523
                    -
                    448
                        -
                        -
    199
        221
            382
                -
                -
            220
                -
                -
        78
            82
                -
                -
            45
                47
                    -
                    -
                -
405
    891
        938
            -
            -
        703
            765
                790
                    -
                    -
                764
                  

### Упражнение 2. Печать  дерева

Добавьте в класс `BST` метод, печатающий дерево согласно алгоритму in-order traverse. Узел с уровнем `4` должен печататься с отступом в `4*n` пробела. Сначала печатается корень поддерева, затем правое поддерево.

## <font color=green>Вставка узла в АВЛ-дерево. Удаление узла из АВЛ-дерева</font>

Алгоритмы состоят из двух этапов.

1. Обычные вставка и удаление, используемые в двоичных деревьях поиска

2. Ретрейсинг. Проходим по дерево снизу вверх, балансируя дерево.
  
  - Для выполнения этой операции в каждый узел добавляется дополнительный параметр - баланс, который равен разности высот правого и левого дерева.
  
  - В процессе движения вверх исправляется баланс и дерево балансируется. Для балансировки достаточно выполнить одну операцию малого поворота (Rotate-Left, Rotate-Right) или одну операцию большого поворота (Rotate-Right-Left, Rotate-Left-Right)

### Пример 3. Вставка в АВЛ-дерево